# Parking-Zone-Prediction


In [336]:
import pandas as pd

# Load the dataset
data = pd.read_csv("/content/drive/MyDrive/2021-2025 -Mini_project/extended_chennai_data (7).csv")

# Display the first few rows of the dataset
print(data.head())

# Check for any missing values
print(data.isnull().sum())

# Ensure data types are correct
print(data.dtypes)


   Latitude  Longitude  Crime Rate  Security Presence  Traffic Density  \
0   13.0827    80.2707           5                  1                2   
1   13.0674    80.2376          20                  0                3   
2   13.0358    80.2449           3                  1                1   
3   13.0846    80.2179           7                  0                2   
4   13.0903    80.2125          25                  0                3   

   Lighting Quality    Zone Type  
0                 3    Safe Zone  
1                 1  Danger Zone  
2                 2    Safe Zone  
3                 3    Safe Zone  
4                 1  Danger Zone  
Latitude             0
Longitude            0
Crime Rate           0
Security Presence    0
Traffic Density      0
Lighting Quality     0
Zone Type            0
dtype: int64
Latitude             float64
Longitude            float64
Crime Rate             int64
Security Presence      int64
Traffic Density        int64
Lighting Quality       int

In [338]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report


In [339]:
X = data[['Latitude', 'Longitude']]
y = data['Zone Type']

In [340]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [341]:
# Define a pipeline that scales the data and trains the KNN model
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('knn', KNeighborsClassifier())
])




In [342]:
# Define hyperparameters for grid search
param_grid = {
    'knn__n_neighbors': [3, 5, 7, 9],
    'knn__weights': ['uniform', 'distance'],
    'knn__algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
}



In [343]:
# Perform grid search with cross-validation
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)



GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('knn', KNeighborsClassifier())]),
             param_grid={'knn__algorithm': ['auto', 'ball_tree', 'kd_tree',
                                            'brute'],
                         'knn__n_neighbors': [3, 5, 7, 9],
                         'knn__weights': ['uniform', 'distance']},
             scoring='accuracy')

In [344]:
# Best parameters found by grid search
best_params = grid_search.best_params_
print("Best Parameters:", best_params)



Best Parameters: {'knn__algorithm': 'auto', 'knn__n_neighbors': 9, 'knn__weights': 'uniform'}


In [345]:
# Train the model with best parameters
best_pipeline = grid_search.best_estimator_
best_pipeline.fit(X_train, y_train)



Pipeline(steps=[('scaler', StandardScaler()),
                ('knn', KNeighborsClassifier(n_neighbors=9))])

In [346]:
# Predicting on the testing set
predictions = best_pipeline.predict(X_test)



In [348]:
# Calculating accuracy and classification report
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(y_test, predictions))

Accuracy: 0.7407407407407407
Classification Report:
              precision    recall  f1-score   support

 Danger Zone       0.25      0.03      0.05        33
   Safe Zone       0.76      0.97      0.85       102

    accuracy                           0.74       135
   macro avg       0.50      0.50      0.45       135
weighted avg       0.63      0.74      0.66       135



In [349]:
# Function to predict zone type for given latitude and longitude
def predict_zone(latitude, longitude, model):
    # Preparing input data
    input_data = pd.DataFrame({'Latitude': [latitude], 'Longitude': [longitude]})

    # Predicting zone type
    zone_prediction = model.predict(input_data)
    return zone_prediction[0]





In [351]:
# Example usage with user interaction
try:
    latitude = float(input("Enter latitude: "))
    longitude = float(input("Enter longitude: "))

    predicted_zone = predict_zone(latitude, longitude, best_pipeline)
    print(f"The predicted zone for the given inputs is: {predicted_zone}")
except ValueError:
    print("Invalid input. Please enter the correct values.")

Enter latitude: 13.084262
Enter longitude: 80.199680
The predicted zone for the given inputs is: Safe Zone
